In [1]:
# Create a Python connection with SQL database and retrieve the results of the last two queries 
#(also mentioned below) as dataframes:

# - Check the number of rentals for each customer for May
# - Check the number of rentals for each customer for June

#   **Hint**: You can store the results from the two queries in two separate dataframes.

# Write a function that checks if customer borrowed more or less films in the month of June 
# as compared to May.

#  **Hint**: For this part, you can create a join between the two dataframes created before, 
#using the merge function available for pandas dataframes. Here is a link to the documentation
#for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [10]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

data_may = pd.read_sql_query('''SELECT customer_id,count(rental_id) FROM rentals_may 
                             group by customer_id''', engine)
data_may.head()

,customer_id,count(rental_id)
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5


In [7]:
data_june = pd.read_sql_query('''SELECT customer_id,count(rental_id) FROM rentals_june 
                             group by customer_id''', engine)
data_june.head()

,customer_id,count(rental_id)
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6


In [13]:
data_rental_merge = data_may.merge(data_june, how='outer', on='customer_id')
data_rental_merge.head()

,customer_id,count(rental_id)_x,count(rental_id)_y
0,130,2.0,6.0
1,459,1.0,7.0
2,408,3.0,3.0
3,333,1.0,4.0
4,222,5.0,2.0


In [27]:
data_rental_merge.columns

Index(['count(rental_id)_x', 'count(rental_id)_y'], dtype='object')

In [23]:
import numpy as np


data_rental_merge.set_index('customer_id', inplace = True)

print(data_rental_merge)


             count(rental_id)_x  count(rental_id)_y
customer_id                                        
130                         2.0                 6.0
459                         1.0                 7.0
408                         3.0                 3.0
333                         1.0                 4.0
222                         5.0                 2.0
...                         ...                 ...
598                         NaN                 1.0
487                         NaN                 1.0
555                         NaN                 1.0
335                         NaN                 1.0
226                         NaN                 2.0

[598 rows x 2 columns]


In [26]:
data_rental_merge['count(rental_id)_x'] = data_rental_merge['count(rental_id)_x'].fillna(0)
data_rental_merge['count(rental_id)_y'] = data_rental_merge['count(rental_id)_y'].fillna(0)
print(data_rental_merge) # replacing the missing entries for customer_id, because they did not
#rent anything in month may or june with 0

             count(rental_id)_x  count(rental_id)_y
customer_id                                        
130                         2.0                 6.0
459                         1.0                 7.0
408                         3.0                 3.0
333                         1.0                 4.0
222                         5.0                 2.0
...                         ...                 ...
598                         0.0                 1.0
487                         0.0                 1.0
555                         0.0                 1.0
335                         0.0                 1.0
226                         0.0                 2.0

[598 rows x 2 columns]


In [36]:
data_rental_merge['RentalJune-RentalMay'] = np.nan

In [38]:
data_rental_merge['RentalJune-RentalMay']= data_rental_merge['count(rental_id)_y']-data_rental_merge['count(rental_id)_x']

In [39]:
data_rental_merge

,count(rental_id)_x,count(rental_id)_y,RentalJune-RentalMay
customer_id,,,
130,2.0,6.0,4.0
459,1.0,7.0,6.0
408,3.0,3.0,0.0
333,1.0,4.0,3.0
222,5.0,2.0,-3.0
...,...,...,...
598,0.0,1.0,1.0
487,0.0,1.0,1.0
555,0.0,1.0,1.0


In [60]:
def rental_diff(index):
    import getpass 
    import numpy as np
    password = getpass.getpass()
    connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
    engine = create_engine(connection_string)
    data_may = pd.read_sql_query('''SELECT customer_id,count(rental_id) FROM rentals_may 
                             group by customer_id''', engine)
    data_june = pd.read_sql_query('''SELECT customer_id,count(rental_id) FROM rentals_june 
                             group by customer_id''', engine)
    data_rental_merge = data_may.merge(data_june, how='outer', on='customer_id')
    data_rental_merge.set_index('customer_id', inplace = True)
    data_rental_merge['count(rental_id)_x'] = data_rental_merge['count(rental_id)_x'].fillna(0)
    data_rental_merge['count(rental_id)_y'] = data_rental_merge['count(rental_id)_y'].fillna(0)
    data_rental_merge['RentalJune-RentalMay'] = np.nan
    data_rental_merge['RentalJune-RentalMay']= data_rental_merge['count(rental_id)_y']-data_rental_merge['count(rental_id)_x']
    return("The customer borrowed in June compared to May this number of movies more:",int(data_rental_merge.at[index, 'RentalJune-RentalMay']))



In [61]:
rental_diff(222) #applying for one customer_id

········


('The customer borrowed in June compared to May this number of movies more:',
 -3)